In [ ]:
%pip install xhtml2pdf

In [26]:
from IPython.display import display, HTML
from xhtml2pdf import pisa 
from dotenv import load_dotenv
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from os import getenv
import pandas as pd

In [15]:
load_dotenv()

True

In [16]:
db_host = getenv('DB_HOST')
db_port = getenv('DB_PORT')
db_user = getenv('DB_USER')
db_password = getenv('DB_PASSWORD')
db_name = getenv('DB_NAME')
group_user = getenv('GROUP_USER')
group_user_pass = getenv('GROUP_USER_PASS')

In [17]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [20]:
con = engine.connect()

In [27]:
def get_current_date() -> str:
   """
   Returns the current date (note :currently set as yesterdays date)
   """
   current_date = str(datetime.now().date() - timedelta(days = 1))
   return current_date
current_date = get_current_date()
current_date

'2022-10-07'

### Quering database

#### Number of rides completed in the past day

In [35]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT COUNT(*) AS number_of_rides 
FROM rides 
WHERE start_date = '{current_date}';
"""
number_of_rides = pd.read_sql_query(query, con)
number_of_rides

,number_of_rides
0,6


#### Gender split of riders of the past day

In [34]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT * 
FROM yusra_stories_production.users
JOIN todays_rides
USING (user_id)
WHERE start_date = '{current_date}';
"""
riders_gender_split = pd.read_sql_query(query, con)
riders_gender_split

,user_id,name,gender,date_of_birth,age,height_cm,weight_kg,address,email_address,account_created,...,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules,start_date
0,4651,Mohammad Phillips-Simpson,male,1984-07-29,38.0,152.0,55.0,"Studio 93s,Mitchell brook,Wilsonmouth,S5U 6EQ",mohammad.phillips-simpson@gmail.com,2022-02-26,...,2022-10-07 15:25:57,2022-10-07 15:34:38,0:08:40,155,80,121,41,50,13.93,2022-10-07
1,4651,Mohammad Phillips-Simpson,male,1984-07-29,38.0,152.0,55.0,"Studio 93s,Mitchell brook,Wilsonmouth,S5U 6EQ",mohammad.phillips-simpson@gmail.com,2022-02-26,...,2022-10-07 15:17:14,2022-10-07 15:25:55,0:08:40,151,85,105,41,48,12.75,2022-10-07
2,4651,Mohammad Phillips-Simpson,male,1984-07-29,38.0,152.0,55.0,"Studio 93s,Mitchell brook,Wilsonmouth,S5U 6EQ",mohammad.phillips-simpson@gmail.com,2022-02-26,...,2022-10-07 15:08:31,2022-10-07 15:17:13,0:08:40,157,71,107,41,47,18.01,2022-10-07
3,4653,Abbie Jones,female,1998-10-11,23.0,188.0,67.0,"561 Freeman underpass,Ahmedhaven,N0 8EL",abbie_j@gmail.com,2021-12-23,...,2022-10-07 15:49:19,2022-10-07 15:58:00,0:08:40,157,67,114,39,41,8.53,2022-10-07
4,4653,Abbie Jones,female,1998-10-11,23.0,188.0,67.0,"561 Freeman underpass,Ahmedhaven,N0 8EL",abbie_j@gmail.com,2021-12-23,...,2022-10-07 15:34:40,2022-10-07 15:43:21,0:08:40,158,80,111,40,41,7.29,2022-10-07
5,4653,Abbie Jones,female,1998-10-11,23.0,188.0,67.0,"561 Freeman underpass,Ahmedhaven,N0 8EL",abbie_j@gmail.com,2021-12-23,...,2022-10-07 15:43:22,2022-10-07 15:49:17,0:05:54,168,81,133,44,44,14.56,2022-10-07


#### Ages of the riders of the past day

#### Average power and heart rate of riders of past day

### Create the HTML Template

In [32]:
def report_block_template( graph_relative_path, caption=''):

    graph_block = (''
            '<a href="{graph_relative_path}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" img src="{graph_relative_path}">'
            '</a>')

    report_block = ('' +
        graph_block +
        '{caption}' + 
        '<br>'      + 
        '</a>' +
        '<br>' +
        '<hr>')                       
    report_layout = (
       '<h2>Deloton Exercise Bikes Daily Report</h2>'
       + '<hr>'
       + report_block
    )
    return report_layout.format(graph_relative_path=graph_relative_path, caption=caption)


graph_relative_path = 'age_distr.png'

report = report_block_template(graph_relative_path, caption='Dis a graph')


In [33]:
display(HTML(report))

### Convert the HTML to PDF

In [28]:

def convert_html_to_pdf(source_html, output_filename):

    result_file = open(output_filename, "w+b")

    pisa_status = pisa.CreatePDF(
            source_html,           
            dest=result_file)           

    result_file.close()           

    return pisa_status.err

In [29]:
convert_html_to_pdf(report, 'report.pdf')

0

In [30]:
! open report.pdf